The more complex version of this would use this pygithub package as shown in here:
- https://python.langchain.com/docs/integrations/toolkits/github

But let's try a simpler version that just sets up a bunch of python functions to automate different github actions like add, commit and pull requests.

Let's start with a simple commit to a branch of some repository.

First, I'll create a github repository using some simple commands.

In [ ]:
# git init -b main

# git add . && git commit -m "Some commit"

# gh repo create

Now, let's try asking an agent to write come code and commit the resulting code to the current branch of this repository.

To do that, let's give the agent the necessary tools it will need which in this case will be python functions that perform different github actions using the `subprocess` package.

We'll follow the basic steps for building a langchain agent:

# Steps for building a simple agent:
- Set up the LLM
- Define the tool or toolkit (list of tools)
- Set up a prompt template
- Connect llm with your tools
- Define your agent as a dict with keys: input, and agent scratchpad 
- Use the Langchain LCEL language to pipe agent, prompt the llm_with_tools variable and the output parser (which can use OpenAI function calling parser)
- Create the agent loop
- Wrap everything into the AgentExecutor
- Invoke the agent with some query input


In [1]:
# setup the llm
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

Now, let's create a tool for the LLM

In [2]:
import subprocess

def github_commit_tool(commit_message="Some commit"):
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

Now! Before we use it with langchain, let's test it by itself.

In [7]:
github_commit_tool("Some testing commit")

[main fd4ef72] Some testing commit
 1 file changed, 45 insertions(+), 1 deletion(-)
branch 'main' set up to track 'origin/main'.


To https://github.com/EnkrateiaLucca/langchain_github_agent.git
 * [new branch]      main -> main


'Committed to Github'

Now, let's take a look at out github repository from the terminal.

![](2023-10-29-20-47-54.png)

Nice! It looks like we are good to go with this first simple tool!

Let's now make it a tool for our langchain agent by adding the @tool. 

In [3]:
from langchain.tools import tool

@tool
def github_commit_tool(commit_message="Some commit"):
    """This function uses the subprocess package to make commits to a github repo pre-defined."""
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

tools = [github_commit_tool]

We added some documentation to our function to abide by the requirements of the tool decorator from langchain.

Now, let's test if a simple agent can use that tool!

Let's start by setting up our prompt template.

In [4]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent_executor = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [5]:
agent_executor.invoke(
    {
        "input": "Make a commit to our github repo with the message: 'testing commits with langchain agent'"
    }
)



> Entering new AgentExecutor chain...
I need to use the github_commit_tool function to make the commit.
Action: github_commit_tool
Action Input: commit_message='testing commits with langchain agent'[main 09206f5] commit_message='testing commits with langchain agent'
 3 files changed, 457 insertions(+), 8 deletions(-)
 create mode 100644 2023-10-29-20-47-54.png
 create mode 100644 basic_langchain_agent.ipynb


To https://github.com/EnkrateiaLucca/langchain_github_agent.git
   fd4ef72..09206f5  main -> main


branch 'main' set up to track 'origin/main'.

Observation: Committed to Github
Thought:The commit has been successfully made.
Final Answer: The commit has been made to our Github repo with the message 'testing commits with langchain agent'.

> Finished chain.


{'input': "Make a commit to our github repo with the message: 'testing commits with langchain agent'",
 'output': "The commit has been made to our Github repo with the message 'testing commits with langchain agent'."}

And voila! We made a commit!

![](2023-10-30-13-04-20.png)

Ok, this is cool but, we can go beyond that, and add the ability to create a file and then commit that file to our desired repository. Let's do that!

To do it all we have to do is amplify the Agent's toolkit with another tool that can create files in the current folder.

In [18]:
def create_file_tool(file_contents):
    with open("file.py", "w") as f:
        f.write(file_contents)
    
    return "File created"

Let's test this function with some contents generated by an llm model:

In [19]:
contents = llm.predict("Write Python code for summing two numbers. Output ONLY THE CODE")

Now, let's get the contents from the output of the llm and feed that to the agent along with a file name.

In [21]:
create_file_tool(contents)

'File created'

Let's check the file was created and can be run from terminal

In [24]:
!ls *.py

file.py


In [23]:
!python ./file.py

15


Perfect! Now, let's add this to our github agent

In [25]:
@tool
def create_file_tool(file_contents):
    """This function creates a file with the contents and file path provided as inputs."""
    with open("file.py", "w") as f:
        f.write(file_contents)
    
    return "File created"

Now, let's add that functionality to our agent by just updating the tools list.

In [26]:
tools = [github_commit_tool, create_file_tool]

Now, let's modify our agent with its new toolkit

In [27]:
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

Now, let's try using both tools in a call to see if the agent can do something slightly more complex now.

In [28]:
agent_executor.invoke({
    "input": "Write Python code for summing two numbers. Output ONLY THE CODE and save that to a file then commit it to our github repository."
})



> Entering new AgentExecutor chain...
I need to use the create_file_tool function to create a file with the Python code for summing two numbers. Then I can use the github_commit_tool function to commit the file to our github repository.
Action: create_file_tool
Action Input: file_contents = "def sum_numbers(a, b):\n    return a + b"
Observation: File created
Thought:Now I need to use the github_commit_tool function to commit the file to our github repository.
Action: github_commit_tool
Action Input: commit_message = "Added code for summing two numbers"[main 02c30b8] commit_message = "Added code for summing two numbers
 3 files changed, 205 insertions(+), 101 deletions(-)
 create mode 100644 2023-10-30-13-04-20.png
 create mode 100644 file.py


To https://github.com/EnkrateiaLucca/langchain_github_agent.git
   09206f5..02c30b8  main -> main


branch 'main' set up to track 'origin/main'.

Observation: Committed to Github
Thought:I now know the final answer
Final Answer: The Python code for summing two numbers has been saved to a file and committed to our github repository.

> Finished chain.


{'input': 'Write Python code for summing two numbers. Output ONLY THE CODE and save that to a file then commit it to our github repository.',
 'output': 'The Python code for summing two numbers has been saved to a file and committed to our github repository.'}

And voila again!
![](2023-10-30-13-15-38.png)

Ok! This is awesome right? Let's just stop for a second and appreciate the simplicity of this implementation and the power that it gives us! Now, on the other hand, we are using a [ZeroShotReactAgent](https://python.langchain.com/docs/modules/agents/agent_types/react#:~:text=Using-,zeroshotreactagent), which is limited when it comes to multi input functions as tools. However, we can easily expand this by leveraging Langchain's amazing [Multi-Input Tools](https://python.langchain.com/docs/modules/agents/tools/multi_input_tool#:~:text=On%20this%20page-,Multi-Input%20Tools,with%20the%20StructuredTool%20class.,-import%20os).

Let's do that to modify our current tool: `create_file_tool()` to accept the file's contents as well as a file path, this way we are not restricted in creating just one type of file with a pre-specified name.

In [29]:
from langchain.tools import StructuredTool

def create_file_tool(file_contents: str, file_path: str) -> str:
    """
    Function that creates a file with the contents 
    and file path provided as inputs.
    """
    with open(file_path, "w") as f:
        f.write(file_contents)
    
    return "File created"

# Let's test it first!
create_file_tool("print('Hello World')", "print_file.py")

'File created'

In [30]:
!ls *.py

file.py       print_file.py


In [32]:
!python print_file.py

Hello World


Great! Now, let's make it a tool for the agent.

In [33]:
create_file_tool = StructuredTool.from_function(create_file_tool)

And, again, let's update our toolkit and our agent.

In [34]:
tools = [github_commit_tool, create_file_tool]

In [35]:
agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

Now, let's test it by asking it to write a simple `README.md` file, giving the agent some context about the project's repo.

In [36]:
agent_executor.invoke({
    "input": "Write a README.md file with a simple description of this project, which is about an introduction to langchain, then commit it to our github repo."
})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "create_file_tool",
  "action_input": {
    "file_contents": "# Introduction to Langchain\n\nThis project is an introduction to Langchain, a programming language designed for blockchain development.\n",
    "file_path": "README.md"
  }
}
```
Observation: File created
Thought:Now that the README.md file has been created, I can commit it to the GitHub repository.
Action:
```
{
  "action": "github_commit_tool",
  "action_input": {
    "commit_message": "Added README.md file"
  }
}
```[main f2c61cf] Added README.md file
 4 files changed, 214 insertions(+), 5 deletions(-)
 create mode 100644 2023-10-30-13-15-38.png
 create mode 100644 README.md
 create mode 100644 print_file.py


To https://github.com/EnkrateiaLucca/langchain_github_agent.git
   02c30b8..f2c61cf  main -> main


branch 'main' set up to track 'origin/main'.

Observation: Committed to Github
Thought:The README.md file has been created and committed to the GitHub repository.

> Finished chain.


{'input': 'Write a README.md file with a simple description of this project, which is about an introduction to langchain, then commit it to our github repo.',
 'output': 'The README.md file has been created and committed to the GitHub repository.'}

And again we succeed!

![](2023-10-30-13-26-32.png)

Ok, before we finish with this example, let's add a last tool to read files from a repo, for situations where we might write some Python code and we want our agent to integrate that Python code into our repository. 

Again, we follow the same procedure:

- Create the tool
- Update our tookit
- Update our agent
- Run a test task to check if it works 

In [37]:
def read_file_tool(file_path):
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

read_file_tool("README.md")

'# Introduction to Langchain\n\nThis project is an introduction to Langchain, a programming language designed for blockchain development.\n'

Ok, our function seems to work so let's add it to our toolkit, since its a simple function with only one input we can just add the @tool decorator on top of it.

In [38]:
@tool
def read_file_tool(file_path):
    """
    Simple function to read contents from a file.
    """
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

And we update our toolkit:

In [39]:
tools = [github_commit_tool, create_file_tool, read_file_tool]

And we update our agent

In [40]:
agent_executor = initialize_agent(tools, llm, 
                                  agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                                  verbose=True
                                  )

Before we run our newly updated agent, let's create a .txt file containing the contents from this notebook so that our agent has something useful to read.

In [45]:
agent_executor.invoke(
    {
    "input": "Update our README.md file based on the contents of the `repo_contents.txt` file, then commit the changes to github repo."
    }
                      )



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "read_file_tool",
  "action_input": {
    "file_path": "repo_contents.txt"
  }
}
```
Observation: The more complex version of this would use this pygithub package as shown in here:
- https://python.langchain.com/docs/integrations/toolkits/github

But let's try a simpler version that just sets up a bunch of python functions to automate different github actions like add, commit and pull requests.
Let's start with a simple commit to a branch of some repository.
First, I'll create a github repository using some simple commands.
# git init -b main

# git add . && git commit -m "Some commit"

# gh repo create
Now, let's try asking an agent to write come code and commit the resulting code to the current branch of this repository.

To do that, let's give the agent the necessary tools it will need which in this case will be python functions that perform different github actions using the `subprocess` package.
We'll follow the bas

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4517 tokens. Please reduce the length of the messages.